# How do we make software understand language?

## Download Dutch dataset

In [ ]:
import os

In [ ]:
if not os.path.isfile("dutch.csv"): 
  !wget https://raw.githubusercontent.com/tijsg/lm-tutorial/main/dutch.csv

## Analogy with language tests
Fill in the missing word

## Encoding
Every string is already encoded character per character

In [ ]:
for char in ("abcdefghijklmnopqrstuvwxyz"):
  print("UTF-8 code for char " + char + " is: " + str(ord(char)))

UTF-8 code for char a is: 97
UTF-8 code for char b is: 98
UTF-8 code for char c is: 99
UTF-8 code for char d is: 100
UTF-8 code for char e is: 101
UTF-8 code for char f is: 102
UTF-8 code for char g is: 103
UTF-8 code for char h is: 104
UTF-8 code for char i is: 105
UTF-8 code for char j is: 106
UTF-8 code for char k is: 107
UTF-8 code for char l is: 108
UTF-8 code for char m is: 109
UTF-8 code for char n is: 110
UTF-8 code for char o is: 111
UTF-8 code for char p is: 112
UTF-8 code for char q is: 113
UTF-8 code for char r is: 114
UTF-8 code for char s is: 115
UTF-8 code for char t is: 116
UTF-8 code for char u is: 117
UTF-8 code for char v is: 118
UTF-8 code for char w is: 119
UTF-8 code for char x is: 120
UTF-8 code for char y is: 121
UTF-8 code for char z is: 122


## Install dependencies

### Check CUDA version

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


### Install dependencies


In [6]:
!pip3 install tokenizers==0.12.1 transformers==4.21.3 pandas==1.5.3 datasets==2.4.0 tensorflow==2.12
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 120.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00
INFO: pip is lo

## Load in data

In [ ]:
import pandas as pd

In [ ]:
df_dutch = pd.read_csv("dutch.csv", delimiter=";")

In [ ]:
df_dutch

,labels
0,Spanje is met ingang van vandaag voorzitter va...
1,Vijf werknemers van het omstreden Amerikaanse ...
2,Het Oud en Nieuwfeest op het Museumplein in Am...
3,President Obama heeft de eerste rapporten gekr...
4,In de hele wereld is het nieuwe jaar feestelij...
...,...
9973,Een Chinese trein heeft het snelheidsrecord ge...
9974,Een universiteit in de Amerikaanse staat Texas...
9975,In Brussel demonstreren tienduizenden mensen v...
9976,De NS wil het papieren spoorboekje afschaffen....


## Split by space

In [ ]:
sentence = df_dutch.iloc[0]
sentence = sentence[0]

In [ ]:
sentence.split(" ")[:10]

['Spanje',
 'is',
 'met',
 'ingang',
 'van',
 'vandaag',
 'voorzitter',
 'van',
 'de',
 'EU.']

## Tokenizers 
### Byte pair encoding

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files="dutch.csv", vocab_size=50265, min_frequency=3, 
                show_progress=True,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

In [ ]:
import os

if not os.path.exists("tokenizer"): os.mkdir("tokenizer")
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("tokenizer", max_len=512)

Didn't find file tokenizer/tokenizer.json. We won't load it.
Didn't find file tokenizer/added_tokens.json. We won't load it.
Didn't find file tokenizer/special_tokens_map.json. We won't load it.
Didn't find file tokenizer/tokenizer_config.json. We won't load it.
loading file tokenizer/vocab.json
loading file tokenizer/merges.txt
loading file None
loading file None
loading file None
loading file None


In [ ]:
tokens = tokenizer.encode(sentence[:100])
print(sentence[:100])
print(tokens)

Spanje is met ingang van vandaag voorzitter van de EU. De Zweedse premier Fredrik Reinfeldt heeft he
[0, 17121, 321, 351, 8590, 284, 880, 2079, 284, 268, 2025, 18, 328, 4668, 1070, 10555, 1137, 17786, 402, 425, 2]


In [ ]:
print(tokenizer.decode([0, 55, 84, 264, 546, 321, 351, 289, 75, 466, 284, 880, 329, 844, 472, 284, 268, 385, 57, 18, 328, 437, 461, 72, 347, 304, 848, 529, 300, 72, 86, 394, 426, 73, 266, 74, 389, 88, 402, 425, 2]))

<s>Spanje is met ingang van vandaag voorzitter van de EU. De Zweedse premier Fredrik Reinfeldt heeft he</s>


## Configure RoBERTa model

This is defined by a research team

In [ ]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

# Set a configuration for our RoBERTa model
config = RobertaConfig(
    vocab_size=50265,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize the model from a configuration without pretrained weights
model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  46730841


### Data collator to mask words with 15 percent probability

In [ ]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15,
)

### Check if a GPU is available

In [ ]:
import torch

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    c = torch.cuda.device_count()
    print(f"CUDA is available!\nNumber of gpu's detected: {c}" )
    print('Training on GPU ...\n\n')
    for i in range(c):
        print(f'Device name {i}== {torch.cuda.get_device_name(i)}')

CUDA is available!
Number of gpu's detected: 1
Training on GPU ...


Device name 0== Tesla T4


### Set up a tokenizer function to prepare for on-the-fly tokenizing

In [12]:
def encode(batch):
    return tokenizer(batch['labels'], padding="max_length", truncation=True, max_length=512,return_tensors="pt")

### Load in the dataset, this time using Hugging Face's "datasets" instead of "pandas"

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="dutch.csv", delimiter=";")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels'],
        num_rows: 9978
    })
})

### Shuffle the dataset, and do a train/test split with 70/30 ratio

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset = dataset["train"]
dataset = dataset.train_test_split(train_size=0.7)

### Check if data is loaded correctly

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels'],
        num_rows: 6984
    })
    test: Dataset({
        features: ['labels'],
        num_rows: 2994
    })
})

In [ ]:
dataset["train"][100]

{'labels': "In RoemenieÌˆ zijn de lichamen van oud-dictator Nicolae Ceausescu en zijn vrouw Elena opgegraven. Dat gebeurt op verzoek van hun kinderen, die zeker willen weten dat de lichamelijke resten die van hun ouders zijn. De Ceausescu's werden eind 1989 na de val van het communistische regime na een ultrakort proces gefusilleerd. Ze werden meteen daarna begraven op een legerbasis in West-Boekarest. De familie wil nu via dna-onderzoek uitgezocht zien of de Ceausescu's daar werkelijk zijn begraven. Als dat niet het geval is, overweegt de familie een proces tegen de staat."}

In [ ]:
dataset["test"][100]

{'labels': 'Olieconcern BP heeft vanwege het olielek in de Golf van Mexico tot nu toe 145.000 schadeclaims ontvangen. Het grootste deel daarvan is al uitbetaald. Dat heeft het bedrijf 310 miljoen dollar gekost, zegt BP. BP houdt er rekening mee dat de uiteindelijke kosten kunnen oplopen tot meer dan 20 miljard dollar. Het olielek in de Golf van Mexico werd op 15 juli gedicht. Op dit moment wordt er een nieuwe gat geboord om de druk op de afgedichte bron te verlichten. '}

### Set up on-the-fly tokenizing/encoding

In [ ]:
ds_train = dataset["train"]
ds_train.set_transform(encode)
ds_test = dataset["test"]
ds_test.set_transform(encode)

Now when fetching a sentence, it returns the tokens instead of the sentence

In [ ]:
ds_train[20]

{'input_ids': tensor([    0,  1513,  6178,   284,  3273,  1434,   991,   268, 10320,    70,
           382,   320,  1396,   319, 11984,   289, 20346,   596, 13844,  8779,
            18,   328,   540,   992,  1370, 27429,   309,   293,  2555,   284,
           268,  3031,    18,  1309,   268,  6572,   284,   293, 12896,   351,
         38441,   307, 30988,   734,  3539,  3607,   284,   268, 39907,  5777,
            18,   710,  1856,  1365,  3826,  1288,   718,   444,  1034,    16,
           756,   268, 16292, 17776,   549,  4965,   319,   293, 12896, 41338,
           439,    18,   328,  3031,  2212,   554,   268,   540,   995,   607,
           319,   430,   268, 45791,   549,  1218,   307,   268,   299,  7097,
          1261,   549,  8892,    18,   553,   644,   319,   430,   319,     2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 

### Set up the training parameters

In [ ]:
from transformers import TrainingArguments, Trainer

if not os.path.exists("models"): os.mkdir("models")
if not os.path.exists("models/roberta"): os.mkdir("models/roberta")

training_args = TrainingArguments(
    output_dir='./models/roberta',
    overwrite_output_dir=True,
    evaluation_strategy = 'steps',
    num_train_epochs=5,
    learning_rate=1e-5,
    lr_scheduler_type="constant",
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=500,
    eval_steps=500,
    save_total_limit=3,
    ignore_data_skip=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    #prediction_loss_only=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


### Start the training script

In [ ]:
trainer.train()
trainer.save_model("models/roberta/roberta_dutch")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6984
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 1090


Step,Training Loss,Validation Loss
500,8.485100,7.429892
1000,7.262300,7.191267


***** Running Evaluation *****
  Num examples = 2994
  Batch size = 8
Saving model checkpoint to ./models/roberta/checkpoint-500
Configuration saved in ./models/roberta/checkpoint-500/config.json
Model weights saved in ./models/roberta/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [models/roberta/checkpoint-600] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2994
  Batch size = 8
Saving model checkpoint to ./models/roberta/checkpoint-1000
Configuration saved in ./models/roberta/checkpoint-1000/config.json
Model weights saved in ./models/roberta/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to models/roberta/roberta_dutch
Configuration saved in models/roberta/roberta_dutch/config.json
Model weights saved in models/roberta/roberta_dutch/pytorch_model.bin


# Load the model
## Load in the newly trained model

In [ ]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM.from_pretrained("models/roberta/roberta_dutch")

loading configuration file models/roberta/roberta_dutch/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 1,
  "num_hidden_layers": 1,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file models/roberta/roberta_dutch/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForMaskedLM.

All the weights of RobertaForMaskedLM were initialized from the model checkpoint at models/roberta/roberta_dutch.
If your t

## Set up a script to check the model with masked words

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model = model,
    tokenizer = tokenizer
)

### Test the model

In [ ]:
fill_mask("Hij gaat <mask>")

[{'score': 0.046948909759521484,
  'token': 284,
  'token_str': ' van',
  'sequence': 'Hij gaat van'},
 {'score': 0.034530047327280045,
  'token': 268,
  'token_str': ' de',
  'sequence': 'Hij gaat de'},
 {'score': 0.02572978101670742,
  'token': 18,
  'token_str': '.',
  'sequence': 'Hij gaat.'},
 {'score': 0.023167967796325684,
  'token': 289,
  'token_str': ' in',
  'sequence': 'Hij gaat in'},
 {'score': 0.0205739326775074,
  'token': 299,
  'token_str': ' een',
  'sequence': 'Hij gaat een'}]

# Model finetuning

## Load in data for sentiment analysis task

### Download annotations

In [2]:
import os
if not os.path.isfile("sentiment.json"): 
  !wget https://raw.githubusercontent.com/tijsg/lm-tutorial/main/sentiment.json

--2023-05-24 08:58:13--  https://raw.githubusercontent.com/tijsg/lm-tutorial/main/sentiment.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 371 [text/plain]
Saving to: ‘sentiment.json’

sentiment.json      100%[===================>]     371  --.-KB/s    in 0s      

2023-05-24 08:58:13 (18.8 MB/s) - ‘sentiment.json’ saved [371/371]



### Load annotations

In [18]:
from datasets import load_dataset
ds_sentiment = load_dataset("json", data_files=["sentiment.json"], field="data")

  0%|          | 0/1 [00:00<?, ?it/s]

4 annotations in dataset

In [19]:
ds_sentiment

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 4
    })
})

In [20]:
ds_sentiment = ds_sentiment["train"]
ds_sentiment = ds_sentiment.train_test_split(train_size=0.7)

In [21]:
ds_sentiment

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2
    })
})

#### Set up on-the-fly tokenizing/encoding

In [22]:
ds_sentiment_train = ds_sentiment["train"]
ds_sentiment_train.set_transform(encode)
ds_sentiment_test = ds_sentiment["test"]
ds_sentiment_test.set_transform(encode)

### Load in base model, this time for classification instead of filling a mask

There are two labels: 0 and 1 (negative and positive)

In [ ]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
model = RobertaForSequenceClassification.from_pretrained("models/roberta/roberta_dutch", num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="../models/sentiment_analysis",
    learning_rate=1e-5,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_steps=200,
    eval_steps=500,
    save_total_limit=3,
    report_to=None,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_sentiment_train,
    eval_dataset=ds_sentiment_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)